<a href="https://colab.research.google.com/github/artanebibi/datascience/blob/main/Sentiment_Analysis_biznisinfo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install selenium

In [ ]:
!pip install python-dotenv


In [ ]:
import pandas as pd
import transformers
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from sklearn.model_selection import train_test_split
import selenium as sns
from queue import Queue
import threading
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.common.exceptions import WebDriverException
from datetime import datetime

Freezing the un-needed parameters from fin-bert model to fasten up the training process. While focusing only on fine-tuning the already trained finBERT model

 <h1> STOCK NAMES AND CODES </h1>

In [ ]:
stocks = [
    # "Komercijalna banka AD Skopje", "Alkaloid AD Skopje", "DS Smith AD Skopje",
    # "Fersped AD Skopje", "Granit AD Skopje", "Hoteli Metropol Ohrid",
    # "Internesnel Hotels AD Skopje", "Makedonijaturist AD Skopje",
    # "Makosped AD Skopje",

    "Makoteks AD Skopje", "Makpetrol AD Skopje",
    "Makstil AD Skopje", "Replek AD Skopje", "RZ Inter-Transsped AD Skopje",
    "RZ Uslugi AD Skopje", "Skopski Pazar AD Skopje", "Stopanska banka AD Bitola",
    "Teteks AD Tetovo", "TTK Banka AD Skopje", "Tutunski kombinat AD Prilep",
    "Vitaminka AD Prilep", "VV Tikves AD Kavadarci", "Zito Luks AD Skopje",
    "ZK Pelagonija AD Bitola", "Ading AD  Skopje", "Agromehanika AD Skopje",
    "Angropromet Tikvesanka AD Kavadarci",
    "ArcelorMittal (HRM) AD Skopje - dolgorocno suspendirano od kotacija",
    "Automakedonija AD Skopje", "BIM AD Sveti Nikole",
    "Blagoj Tufanov AD Radovis - dolgorocno suspendirano od kotacija",
    "Cementarnica USJE AD Skopje", "Centralna kooperativna banka AD Skopje",
    "Debarski Bani –Capa AD  Debar", "Dimko Mitrev AD Veles",

    # "Fabrika Karpos AD Skopje", "FAKOM AD Skopje", "Fruktal Mak AD Skopje",
    # "Fustelarko Borec AD Bitola", "FZC 11-ti OKTOMVRI AD Kumanovo",
    # "GD-TIKVES AD Kavadarci", "Geras Cunev Konfekcija AD Strumica",
    # "Geras Cunev Trgovija AD Strumica", "Grozd AD Strumica", "GTC AD Skopje",
    # "Interpromet AD Tetovo", "Klanica so ladilnik AD Strumica",
    # "Kristal 1923 AD Veles", "Liberti AD Skopje", "Lotarija na Makedonija AD Skopje",
    # "Makedonija osiguruvane AD Skopje - Viena Insurens Grup",
    # "Makedonski Telekom AD – Skopje", "Makpromet AD Stip",
    # "Mermeren kombinat AD Prilep", "Moda AD Sveti Nikole", "MZT Pumpi AD Skopje",
    # "Nemetali Ograzden AD Strumica", "NLB Banka AD Skopje",
    # "Nova Stokovna kuka AD Strumica", "OILKO KDA Skopje", "OKTA AD Skopje",
    # "Oranzerii Hamzali Strumica", "Patnicki soobrakaj Transkop AD Bitola",
    # "Pekabesko AD Kadino Ilinden", "Pelisterka AD Skopje", "Popova Kula AD Demir Kapija",
    # "Prilepska pivarnica AD Prilep", "Rade Koncar- Aparatna tehnika AD Skopje",
    # "Rudnici Banani AD Skopje", "RZ Ekonomika AD Skopje",
    # "RZ Tehnicka Kontrola AD Skopje", "Sigurnosno staklo AD Prilep",
    # "Sileks AD Kratovo", "Slavej  AD Skopje", "Sovremen dom AD Prilep",
    # "Stokopromet AD Skopje", "Stopanska banka AD Skopje",
    # "Strumicko pole s. Vasilevo", "Tajmiste AD Kicevo - dolgorocno suspendirano od kotacija",
    # "TEAL AD  Tetovo", "Tehnokomerc AD Skopje",
    # "Trgotekstil Maloprodazba AD Skopje", "Triglav Osiguruvane AD Skopje",
    # "Trudbenik AD Ohrid - dolgorocno suspendirano od kotacija",
    # "Ugotur AD Skopje", "UNI Banka AD Skopje", "Vabtek MZT AD Skopje",
    # "Veteks AD Veles", "ZAS AD Skopje", "Zito Karaorman AD Kicevo",
    # "Zito Polog AD Tetovo"
]


def filter_stockname():
    stock_names = []
    for company in stocks:
        parts = company.split()
        # get rid of city names for better search
        filtered = [part for part in parts if
                    part not in {"AD", "Skopje", "Bitola", "Tetovo", "Prilep", "Kavadarci", "Ohrid", "Veles",
                                 "Kumanovo", "Sveti", "Nikole", "Debar", "Radovis", "Stip", "Ilinden", "Kicevo",
                                 "Strumica", " - dolgorocno suspendirano od kotacija"}]
        stock_names.append(" ".join(filtered))

    return stock_names


def getStockCode_db():

  stock_codes = [
    # "KMB", "ALK", "KOMU", "FERS", "GRNT", "MPOL", "INHO", "MTUR", "MKSD",
    "MAKS", "MPT", "STIL", "REPL", "RZIT", "RZUS", "SPAZ", "SBT", "TETE", "TTK", "TKPR",
    "VITA", "TKVS", "ZILU", "ZPKO", "ADIN", "AMEH", "APTK", "RZLE", "AUMK", "BIM",
    "BLTU", "USJE", "CKB", "DEBA", "DIMI",
    # "KPSS", "FAKM", "KONZ", "FUBT",
    # "CEVI", "TIKV", "GECK", "GECT", "GRZD", "GTC", "INPR", "KLST", "BGOR", "RZLV",
    # "LOTO", "KJUBI", "TEL", "MAKP", "MERM", "MODA", "MZPU", "NEME", "TNB", "NOSK",
    # "OILK", "OKTA", "ORAN", "TRPS", "PKB", "LOZP", "POPK", "PPIV", "RADE", "BANA",
    # "RZEK", "RZTK", "SSPR", "SIL", "SLAV", "SDOM", "STOK", "STB", "SPOL", "TAJM",
    # "TEAL", "TEHN", "TSMP", "VROS", "TRDB", "RZUG", "UNI", "MZHE", "VTKS", "ZAS",
    # "ZKAR", "ZPOG"
  ]


  return stock_codes



In [ ]:
def create_driver():
    options = webdriver.ChromeOptions()
    options.add_argument("--headless")
    options.add_argument("--disable-dev-shm-usage")
    options.add_argument("--no-sandbox")
    options.add_argument("--disable-gpu")
    return webdriver.Chrome(options=options)


class SeleniumDriverPool:
    def __init__(self, max_size=5):
        self.pool = Queue(maxsize=max_size)
        self.lock = threading.Lock()
        for _ in range(max_size):
            driver = create_driver()
            self.pool.put(driver)

    def acquire_driver(self):
        with self.lock:
            return self.pool.get()

    def release_driver(self, driver):
        with self.lock:
            self.pool.put(driver)

    def close_all_drivers(self):
        while not self.pool.empty():
            driver = self.pool.get()
            driver.quit()


pool = SeleniumDriverPool(max_size=20)

<H1> SCRAPING <i>kapital.mk

In [ ]:
def split_list(lst):
    mid = len(lst) // 2
    return lst[:mid], lst[mid:]

In [ ]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.common.exceptions import WebDriverException
from datetime import datetime
import threading
from queue import Queue
import os


def scrape_kapitalMK():
    stock_dataframes = {}
    filtered_stocks, filtered_stocks_2 = split_list(filter_stockname())
    stock_codes, stock_codes_2 = split_list(getStockCode_db())

    pool1 = SeleniumDriverPool(max_size=20)
    while True:
        for stock_name, stock_code in zip(filtered_stocks, stock_codes):
            data = []

            try:
                driver = pool1.acquire_driver()
                i = 1

                while True:
                    driver.get(f'https://kapital.mk/page/{i}/?s={stock_name}')
                    html_content = driver.page_source

                    if 'Error 404!' in html_content or 'Sorry, your search did not match any entries.' in html_content:
                        break

                    article_list = driver.find_elements(By.CSS_SELECTOR, ".mvp-blog-story-list li")

                    for tag in article_list:
                        a_tag = tag.find_element(By.CSS_SELECTOR, "a:nth-child(1)")
                        link = a_tag.get_attribute('href')
                        print(link)

                        try:
                            nd = pool1.acquire_driver()
                            nd.get(link)
                            date = nd.find_element(By.CSS_SELECTOR, 'time')
                            date_published = datetime.fromisoformat(date.get_attribute('datetime')).strftime("%d.%m.%Y")
                            print()
                            print(date_published)
                            print()
                            post = nd.find_element(By.CSS_SELECTOR, ".mvp-post-soc-out").text

                            actual_post = post.split('ПОВРЗАНИ ТЕМИ:')[0]
                            # print(actual_post)

                            title = a_tag.text
                            print(title)
                            data.append({'date': date_published, 'title': title, 'text': actual_post, 'link': link, 'from': 'kapital.mk'})

                        except WebDriverException as e:
                            print(f"Error occurred while processing link {link}: {e}")

                        finally:
                            pool1.release_driver(nd)

                    i += 1

            except WebDriverException as e:
                print(f"Error occurred for stock {stock_name} on page {i}: {e}")

            finally:
                pool1.release_driver(driver)

            stock_dataframes[stock_code] = pd.DataFrame(data)
        break

    pool1.close_all_drivers()

    return stock_dataframes


news2 = scrape_kapitalMK()

ТЕХНОЛОГИИ / КОМПАНИИ
9 месеци ago
ЕУ има амбиција да стане „квантна долина на светот“, а хрватски научници имаат голем удел во тоа
Моќта на квантните компјутери допрва се испитува
https://kapital.mk/%d0%bc%d0%b0%d1%80%d0%ba%d0%be-%d1%87%d0%b0%d0%b4%d0%b5%d0%b6-%d1%81%d1%80%d0%bf%d1%81%d0%ba%d0%b0-%d1%81%d1%82%d0%be%d0%bf%d0%b0%d0%bd%d1%81%d0%ba%d0%b0-%d0%ba%d0%be%d0%bc%d0%be%d1%80%d0%b0-%d0%bd/

15.04.2024

РЕГИОН
9 месеци ago
Марко Чадеж, Српска стопанска комора: Нашите компании ги мачи финансирањето и недостиг на кадри
Српските компании со слични проблеми како и македонските
https://kapital.mk/%d1%81%d1%83%d0%b4%d0%b1%d0%b8%d0%bd%d0%b0%d1%82%d0%b0-%d0%bd%d0%b0-%d1%84%d0%b0%d0%b1%d1%80%d0%b8%d0%ba%d0%b0%d1%82%d0%b0-%d0%b2%d0%be-%d0%b1%d1%83%d0%bd%d0%b0%d1%80%d1%9f%d0%b8%d0%ba-%d0%bd%d0%b5/

15.04.2024

МАКЕДОНИЈА
9 месеци ago
Судбината на фабриката во Бунарџик неизвесна, но можно е и целото производство да се префрли во Македонија
Ван Хол, белгискиот производител на автобуси банкроти

<h1>SCRAPING <i>biznisinfo.mk

In [ ]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.common.exceptions import WebDriverException
from datetime import datetime
import threading
from queue import Queue
import os



def scrape_biznisinfoMK():
    stock_dataframes = {}
    filtered_stocks, filtered_stocks_2 = split_list(filter_stockname())
    stock_codes, stock_codes_2 = split_list(getStockCode_db())
    pool2 = SeleniumDriverPool(max_size=20)
    while True:

        for stock_name, stock_code in zip(filtered_stocks, stock_codes):
            data = []

            try:
                driver = pool2.acquire_driver()
                i = 1

                while True:
                    driver.get(f'https://biznisinfo.mk/page/{i}/?s={stock_name}')
                    html_content = driver.page_source

                    if 'Упсссс... Error 404' in html_content or 'Нема резултати за вашето пребарување' in html_content:
                        break

                    article_list = driver.find_elements(By.CSS_SELECTOR, ".td_module_10")
                    print(f'Title: {article_list}')
                    for tag in article_list:
                        a_tag = tag.find_element(By.CSS_SELECTOR, "div:nth-child(1) > a:nth-child(1)")
                        link = a_tag.get_attribute('href')
                        print(link)

                        try:
                            nd = pool2.acquire_driver()
                            nd.get(link)

                            date = nd.find_element(By.CSS_SELECTOR, 'time')
                            date_published = datetime.strptime(date.text, "%d/%m/%Y").strftime("%d.%m.%Y")
                            print()
                            print(date_published)
                            print()

                            title = nd.find_element(By.CSS_SELECTOR, "h1.entry-title").text
                            print(title)

                            post = nd.find_element(By.CSS_SELECTOR, ".td-main-content").text
                            actual_post = post.split('Можеби ќе ве интересира и...')[0]

                            # if stock_name in actual_post:
                            data.append({'date': date_published, 'title': title, 'text': actual_post, 'link': link, 'from': 'biznisinfo.mk'})

                        except WebDriverException as e:
                            print(f"Error occurred while processing link {link}: {e}")

                        finally:
                            pool2.release_driver(nd)

                    i += 1

            except WebDriverException as e:
                print(f"Error occurred for stock {stock_name} on page {i}: {e}")

            finally:
                pool2.release_driver(driver)

            stock_dataframes[stock_code] = pd.DataFrame(data)

        break

    pool2.close_all_drivers()
    return stock_dataframes




news1 = scrape_biznisinfoMK()

In [ ]:
for stock_name, df in news2.items():
        print(f"Data for {stock_name}:")
        print(df.to_string())


In [ ]:
!pip install boto3

In [ ]:
import io
import os
import boto3
import pandas as pd
from botocore.exceptions import ClientError
from dotenv import load_dotenv

load_dotenv()


class WasabiClient:
    def __init__(self):
        access_key = 'A02ZUKINN1XDCJKJYI0D'
        secret_key = 'ZB0o901t7NOFzmkmOU34HDFg2K1Pdrr0d3XKRhjh'
        self.bucket =  'mkdstocks'
        self.s3_client = boto3.client(
            's3',
            endpoint_url='https://s3.eu-central-2.wasabisys.com',
            aws_access_key_id=access_key,
            aws_secret_access_key=secret_key,
        )

    def fetch_data(self, key: str):
        cloud_key = f"Stock_Data/{key}.csv"

        try:
            file_response = self.s3_client.get_object(Bucket=self.bucket, Key=cloud_key)
            file_content = file_response['Body'].read().decode('utf-8')  # Fix typo
            return pd.read_csv(io.StringIO(file_content))
        except ClientError as e:
            print(f"Error fetching data: {e}")
            return None

    def update_or_create(self, code: str, new_df: pd.DataFrame):
        cloud_key = f"Stock_Data/{code}.csv"

        try:
            try:
                existing_file = self.s3_client.get_object(Bucket=self.bucket, Key=cloud_key)
                existing_df = pd.read_csv(io.StringIO(existing_file['Body'].read().decode('utf-8')))
            except ClientError as e:
                if e.response['Error']['Code'] == 'NoSuchKey':
                    existing_df = None
                else:
                    print(f"Error fetching existing data: {e}")
                    return False

            combined_df = pd.concat([new_df, existing_df]).drop_duplicates() if existing_df is not None else new_df
            csv_buffer = io.StringIO()
            combined_df.to_csv(csv_buffer, index=False)
            csv_buffer.seek(0)
            self.s3_client.put_object(Bucket=self.bucket, Key=cloud_key, Body=csv_buffer.getvalue())
            print(f"Successfully appended and uploaded data for {code}.")
        except ClientError as e:
            print(f"Error uploading data: {e}")

    def create_articles(self, code: str, df: pd.DataFrame):
        cloud_key = f"Articles/{code}.csv"

        try:
            csv_buffer = io.StringIO()
            df.to_csv(csv_buffer, index=False)
            csv_buffer.seek(0)
            self.s3_client.put_object(Bucket=self.bucket, Key=cloud_key, Body=csv_buffer.getvalue())
            print(f"Successfully appended and uploaded data for {code}.")
        except ClientError as e:
            print(f"Error uploading data: {e}")

    def fetch_articles(self, code: str):
        cloud_key = f"Articles/{code}.csv"
        try:
            file_response = self.s3_client.get_object(Bucket=self.bucket, Key=cloud_key)
            file_content = file_response['Body'].read().decode('utf-8')  # Fix typo
            return pd.read_csv(io.StringIO(file_content))
        except ClientError as e:
            print(f"Error fetching data: {e}")
            return None


def get_wassabi_client():
    global wassabi_client
    if wassabi_client is None:
        raise Exception("WasabiClient is not initialized. Check BackendConfig or initialization logic.")
    return wassabi_client


# Setter for initialization
def set_wassabi_client(client):
    global wassabi_client
    wassabi_client = client


def initialize_wasabi_client():
    return WasabiClient()


wassabi_client = None


# *MODELS AND SENTIMENT ANALYSIS*

In [ ]:
from transformers import MarianMTModel, MarianTokenizer, pipeline, AutoTokenizer, AutoModelForSequenceClassification

tokenizer_finbert = AutoTokenizer.from_pretrained("yiyanghkust/finbert-tone")
model_finbert = AutoModelForSequenceClassification.from_pretrained("yiyanghkust/finbert-tone",
                                                                   output_hidden_states=False)
sentiment_analyzer = pipeline("sentiment-analysis", model="yiyanghkust/finbert-tone")

# for translation
tokenizer_translator = MarianTokenizer.from_pretrained('Helsinki-NLP/opus-mt-mk-en')
model_translator = MarianMTModel.from_pretrained('Helsinki-NLP/opus-mt-mk-en')


In [ ]:
def split_text_into_chunks(text, max_length):
    words = text.split()
    chunks = []
    current_chunk = []

    for word in words:
        if len(' '.join(current_chunk + [word])) <= max_length:
            current_chunk.append(word)
        else:
            chunks.append(' '.join(current_chunk))
            current_chunk = [word]

    if current_chunk:
        chunks.append(' '.join(current_chunk))

    return chunks


wasabi_client = WasabiClient()

for stock_name, df in news2.items():
  final_df = []
  for index, row in df.iterrows():
    # title
    macedonian_title = row['title'].replace('„', '').replace('“', '')
    encoded_input = tokenizer_translator(macedonian_title, return_tensors='pt')
    translated = model_translator.generate(**encoded_input)
    english_title = tokenizer_translator.decode(translated[0], skip_special_tokens=True)

    final_df.append({'date': row['date'], 'title': english_title, 'link': row['link']})

    # content - cant exceed 512 tokens
    macedonian_content = row['text']
    chunks = split_text_into_chunks(macedonian_content, max_length=512)
    translated_chunks = []

    for i, chunk in enumerate(chunks):
      encoded_input2 = tokenizer_translator(chunk, return_tensors='pt')
      translated2 = model_translator.generate(**encoded_input2)
      translated_chunk = tokenizer_translator.decode(translated2[0], skip_special_tokens=True)

      result = sentiment_analyzer(translated_chunk)
      print(result[0]['label'], result[0]['score'])
      print()

      final_df.append({f'Chunk: {i}': translated_chunk, f'Sentiment: {i}': result[0]['label']})

      # Combine the translated chunks
      english_content = ' '.join(translated_chunks)

      # Print translated results
      print(f"Title (English): {english_title}")
      print(f"Content (English): {english_content}\n")



    # Optional: Save the translations to WasabiClient
  final_for_stock = pd.DataFrame(final_df)
  wasabi_client.create_articles(stock_name, final_for_stock)

# Example fetching articles
# wasabi_client.fetch_articles('Alkaloid')
